<a href="https://colab.research.google.com/github/suer00/UczenieMaszynowe_w_Finansach_PaulinaReus/blob/main/UczenieMaszynoweFinanse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as scikit_learn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
#Biblioteka do undersamplingu i oversamplingu
from imblearn.under_sampling import RandomUnderSampler

from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression

In [11]:

df_credit = pd.read_csv('/content/creditcard.csv')
df_credit=df_credit.dropna()
print(len(df_credit[df_credit['Class'] == 0]))
print(len(df_credit[df_credit['Class'] == 1]))


284315
492


In [12]:
df_credit.groupby(['Class']).size()

Class
0    284315
1       492
dtype: int64

In [13]:
df_x= df_credit.drop('Class', axis=1)

In [14]:
X= df_x
y=df_credit['Class']

# using the train test split function
X_train, X_test, y_train, y_test = train_test_split(X,y ,
                                   random_state=104,
                                   test_size=0.25,
                                   shuffle=True)

K najbliższych sąsiadów

In [15]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X, y)


KNeighborsClassifier(n_neighbors=3)

In [16]:
KNeighborsClassifier(...)
print(neigh.predict([[1.1]]))
[0]
print(neigh.predict_proba([[0.9]]))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


ValueError: X has 1 features, but KNeighborsClassifier is expecting 30 features as input.

In [17]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

Accurtency

In [18]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9983567877306817


Precission

In [ ]:
precision_score(y_test, y_pred)

Recall

In [19]:
recall_score(y_test, y_pred)

0.1076923076923077

Confusion matrix

In [20]:
cf_matrix = confusion_matrix(y_test, y_pred)
cf_matrix

array([[71071,     1],
       [  116,    14]])

Losowe usuwanie z próbki

In [ ]:
rus=RandomUnderSampler() #RandomUnderSampler object
x_train_rus,y_train_rus=rus.fit_resample(X_train,y_train)#apply to RandomUnderSample
y_train_rus.value_counts()


Usuwa na podstawie K-najjbliższych sąsiadów i usuwa te najbliższe
Zostawiamy na te które są na granicy decyzji

In [ ]:
from imblearn.under_sampling import NearMiss
undersample = NearMiss(version=3)
undersample = NearMiss(version=3, n_neighbors=3)
# transform the dataset
X_train_nm, y_train_nm = undersample.fit_resample(X_train, y_train)

y_train_nm.value_counts()

Tetsowanie nowej próbki

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_nm, y_train_nm)
y_pred_nm = knn.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred_nm)
recall= recall_score(y_test, y_pred_nm)
precision = precision_score(y_test, y_pred_nm)
print("Accuracy:", accuracy)
print("Precision:",precision)
print("Recall:", recall)

SMOTE - znajduje dane z k-najbliższych sąsiadów i na ich podstawie dorabia dane

In [ ]:
oversample = SMOTE()
X_train_os, y_train_os = oversample.fit_resample(X_train, y_train)
y_train_os.value_counts()


Ponownie testujemy

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_os, y_train_os)
y_pred_os = knn.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred_os)
recall= recall_score(y_test, y_pred_os)
precision = precision_score(y_test, y_pred_os)
print("Accuracy:", accuracy)
print("Precision:",precision)
print("Recall:", recall)

Pipline

Głownie patrzymy na reaccol -ponieważ chcemy  zminimalizować ilość false -nagtive->recalll jak największe

In [ ]:
# transform the dataset
X, y = pipeline.fit_resample(X, y)

In [21]:
model = knn
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)
#steps = [('over', over), ('under', under), ('model', model)]
pipeline = make_pipeline(over, under, model)
# evaluate pipeline
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.93      0.96     71072
           1       0.01      0.53      0.03       130

    accuracy                           0.93     71202
   macro avg       0.51      0.73      0.49     71202
weighted avg       1.00      0.93      0.96     71202



Cost-sensitive logistic regression

In [29]:
# define model
weights = {0:1.0, 1:1.0}
model = LogisticRegression(solver='lbfgs',class_weight=weights)


In [30]:
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight={0: 1.0, 1: 1.0})

In [37]:
y_pred = model.predict(X_test)

In [38]:
accuracy = accuracy_score(y_test, y_pred)
recall= recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Precision:",precision)
print("Recall:", recall)

Accuracy: 0.9686244768405382
Precision: 0.04965753424657534
Recall: 0.8923076923076924


In [39]:
# define model
weights = {0:1.0, 1:1.0}
model = LogisticRegression(solver='lbfgs',class_weight='balanced')


In [44]:
model.fit(X_train, y_train)

LogisticRegression(class_weight='balanced')

In [45]:
y_pred = model.predict(X_test)

In [46]:
accuracy = accuracy_score(y_test, y_pred)
recall= recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Precision:",precision)
print("Recall:", recall)

Accuracy: 0.9686244768405382
Precision: 0.04965753424657534
Recall: 0.8923076923076924


PCA